## Initialize

In [1]:
# import required libraries
import logging
import os
import re
import textwrap
from dataclasses import asdict, dataclass
from datetime import date, datetime
from pathlib import Path

import altair as alt
import numpy as np
import polars as pl
import torch
from datasets import Dataset, load_from_disk
from polars import col as c
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    roc_curve,
)
from sklearn.preprocessing import label_binarize
from tqdm import tqdm
from transformers import AutoTokenizer
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

# Configure logging to show timestamp, log level and message
logging.basicConfig(
    format="[%(asctime)s] [%(levelname)s] %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S %p",
)

work_dir = os.getcwd()
logging.info(f"Working directory: {work_dir}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


[2025-01-27 22:45:27 PM] [INFO] Working directory: /home/yu/chaoyang/projects/Call/call/code/v4/reproduce-finetune


## Configuration

**Inference using the non-finetuned model:**

- set `model_name` to `"unsloth/mistral-7b-instruct-v0.3-bnb-4bit"` for Mistral 7B

- set `model_name` to `"unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"` for Llama 3.1 8B

**Inference using the finetuned model:**
- set `model_name` to the path of the finetuned model, 

    - e.g., `"saved_results/mistral"` for Mistral 7B
    - e.g., `"saved_results/llama"` for Llama 3.1 8B


In [3]:
# `model_name`

# If you want to use the non-finetuned model, please set `model_name` to the following:
# - "unsloth/mistral-7b-instruct-v0.3-bnb-4bit" for Mistral 7B
# - "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit" for Llama 3.1 8B

# If you want to use the finetuned model, please set `model_name` to the path of the finetuned model, 
# e.g., `"saved_results/mistral_22q4"` for Mistral 7B
# e.g., `"saved_results/llama-3.1_22q3"` for Llama 3.1 8B

model_name = "saved_results/llama-3.1_22q3"
max_seq_length = 20000
max_new_tokens = 5

# use finratio or not
use_finratio = True

# path to the test data
test_data_path = "data/test_22q3.feather"

# sanity check if `chat_template` matches `model_name`
# by checking if `llama` or "mistral" appear in both
if "llama" in model_name.lower():
    chat_template = "llama-3.1"
elif "mistral" in model_name.lower():
    chat_template = "mistral"
else:
    raise ValueError("model_name must contain 'llama' or 'mistral'")


# path to saved results
def get_saved_results_path():
    split_id = re.search(r"(\d{2}q\d)", test_data_path).group(1)
    features = "frtxt" if use_finratio else "txt"
    if_finetuned = "noft" if "unsloth" in model_name else "ft"
    saved_results_path = f"saved_results/results_{chat_template}_{if_finetuned}_{features}_{split_id}.feather"
    return saved_results_path


saved_results_path = get_saved_results_path()

## Prepare Data

Import data for training

In [4]:
# read the huggingface dataset
test_data = pl.read_ipc(test_data_path)
logging.info(f"Read data from {test_data_path}")

[2025-01-27 22:45:38 PM] [INFO] Read data from data/test_22q4.feather


Next, we construct prompts using the training and testing data. The output consists of two huggingface datasets: `train_dataset` and `test_dataset`.

In [ ]:
# get tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = get_chat_template(
    tokenizer,
    chat_template=chat_template,
    map_eos_token=True,  # e.g., maps <|im_end|> to </s> instead
)


# construct the prompt for training data
def construct_prompt(dataset, use_finratio):

    output = []
    for row in dataset.iter_rows(named=True):
        # define prompt
        prompt_transcript = textwrap.dedent(f"""
            You are a financial analyst. You will be given an earnings call transcript of a company. It may contain three parts: Management Discussion, Questions from Analysts, and Answers from Management. The "Management Discussion" section is a statement from the management (usually CEOs and CFOs) about the past performance and future prospects of the company. The "Questions from Analysts" part consists of questions from financial analysts, and the "Answers from Management" part contains responses from the management. There may be multiple rounds of questions and answers in a call. Your task is to predict whether the earnings call will have a positive or negative impact on the future stock return. Please answer by typing an integer score in the range of 1 to 5, where 1 is the most negative and 5 is the most positive. Your reply must start with "Score:N". Please do not concentrate your predictions on the same score, i.e., the number of stocks falling into each score should be balanced. Please also note that the management usually uses very positive language when discussing their company, but you should not take it for granted. Pay attention to the questions from the analysts. 
                    
            Now the earnings call transcript begins:
                    
            {row["text"]}
        """).strip()

        prompt_finratio = textwrap.dedent(f"""
            Now the financial ratios begin:
            - Earnings surprise (normalized by stock price): {row["sue3"]}
            - Return volatility in the past month: {row["vol_call_m21_m1"]}
            - Market capitalization (log-transformed): {row["mcap"]}
            - Book-to-market ratio: {row["bm"]}
            - Return-on-assets: {row["roa"]}
            - Debt-to-assets: {row["debt_assets"]}
            - Median earnings forecast: {row["medest"]}
            - Number of analysts forecasting: {row["numest"]}
            - Standard deviation of earnings forecast: {row["stdest"]}
            - Turnover in the past month: {row["turnover_ma21"]}
            - Trading volume in the past month: {row["volume_ma21"]}
        """).strip()

        if use_finratio:
            prompt = prompt_transcript + "\n\n" + prompt_finratio
        else:
            prompt = prompt_transcript

        # define user and assistant messages
        user = {
            "transcriptid": row["transcriptid"],
            "docid": row["docid"],
            "docid_idx": row["docid_idx"],
            "role": "user",
            "content": prompt,
        }

        # create prompt based on finetune_or_inference
        output.append([user])

    # convert to HF Dataset
    dataset = Dataset.from_dict({"chat": output})

    # apply chat template
    dataset_for_inference = dataset.map(
        lambda x: {
            "input_ids": tokenizer.apply_chat_template(
                x["chat"], tokenize=True, add_generation_prompt=True, return_tensors="np", max_length=max_seq_length
            )
        },
        batched=True,
    )
    dataset_for_inference.set_format(type="torch", columns=["input_ids"])

    return dataset_for_inference

# construct the prompt for training and testing data
test_dataset_for_inference = construct_prompt(test_data, use_finratio)

## Inference

First, we load the model.

> Important: 
>
> - If you want to use the finetuned model, please set `model_name=saved_model_name` while you're calling `.from_pretrained()` in the following cell; 
>
> - if you want to use the original model, please set `model_name=model_name`.

In [ ]:
# Get model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=torch.bfloat16,  # None for autodetect
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

Next, we iterate through the testing dataset and generate predictions.

> The generated results will be saved in `saved_results_path`.

In [6]:
results = []
for i, (call, input_ids) in tqdm(
    enumerate(
        zip(test_dataset_for_inference["chat"], test_dataset_for_inference["input_ids"])
    ),
    total=len(test_dataset_for_inference),
):
    # # only predict on the first 10 calls for debugging
    # if i > 10:
    #     break

    transcriptid = call[0]["transcriptid"]
    docid = call[0]["docid"]
    docid_idx = call[0]["docid_idx"]
    prompt = call[0]["content"]
    input_length = input_ids.shape[0]  # input_ids is a 1D tensor

    # debug only
    # if not (input_length >= 5000):
    #     logging.info(f"Skipping \"{docid}\" ({input_length} tokens)")
    #     continue

    generated_ids = model.generate(
        input_ids=input_ids.unsqueeze(0).to(model.device),
        max_new_tokens=max_new_tokens,
        use_cache=True,
        do_sample=True,
    )
    generated_text = tokenizer.batch_decode(
        generated_ids[:, input_length:], skip_special_tokens=True
    )[0]
    print(generated_text)

    # every output saved as a single file
    out = {
        "transcriptid": transcriptid,
        "docid": docid,
        "docid_idx": docid_idx,
        "generated_text": generated_text,
    }
    out = pl.DataFrame(out)
    results.append(out)


# at the end, read all the files and save as a feather file
results = pl.concat(results, how="vertical")

# save the results
results.write_ipc(saved_results_path)

 41%|████      | 478/1161 [09:17<13:31,  1.19s/it]

Score:3


 41%|████▏     | 479/1161 [09:18<13:32,  1.19s/it]

Score:3


 41%|████▏     | 480/1161 [09:19<13:04,  1.15s/it]

...

Score:3


100%|█████████▉| 1160/1161 [22:12<00:01,  1.03s/it]

Score:3


100%|██████████| 1161/1161 [22:13<00:00,  1.15s/it]

Score:2
